In [1]:
import requests

In [2]:
prov = requests.get("https://sdgsdev.kemendesa.go.id/SIDcutoff/prov")

In [11]:
import json

with open("provinsi.json", "w") as x:
    x.write(json.dumps(prov.json(), indent=3))


In [14]:
# read json
import pandas as pd
data = json.load(open("provinsi.json"))
prov = pd.DataFrame(data['data'])

In [33]:
def get_kabupaten(prov: pd.DataFrame):
    lenPr = prov["id"].tolist()
    kab = []
    for i in lenPr:
        data_kab = requests.get("https://sdgsdev.kemendesa.go.id/SIDcutoff/kota/{}".format(i)).json()["data"]
        for j in data_kab:
            j["idProv"] = i
            kab.append(j)
    return kab
        


In [34]:
kab=get_kabupaten(prov)
with open("kabupaten.json", "w") as x:
    x.write(json.dumps(kab, indent=3))



In [37]:
kab = json.load(open("kabupaten.json"))
kab = pd.DataFrame(kab)
kab.head()

,id,value,idProv
0,1103,KAB. ACEH SELATAN,11
1,1104,KAB. ACEH TENGGARA,11
2,1105,KAB. ACEH TIMUR,11
3,1106,KAB. ACEH TENGAH,11
4,1107,KAB. ACEH BARAT,11


In [38]:
def get_kecamatan(kab: pd.DataFrame):
    lenKab = kab["id"].tolist()
    kec = []
    for i in lenKab:
        data_kec = requests.get("https://sdgsdev.kemendesa.go.id/SIDcutoff/kecamatan/{}".format(i)).json()["data"]
        for j in data_kec:
            j["idKab"] = i
            kec.append(j)
    return kec
        

In [39]:
kec = get_kecamatan(kab)
with open("kecamatan.json", "w") as x:
    x.write(json.dumps(kec, indent=3))

In [41]:
kec =pd.DataFrame(kec)

In [42]:
def get_kelurahan(kec: pd.DataFrame):
    lenKec = kec["id"].tolist()
    kel = []
    for i in lenKec:
        data_kel = requests.get("https://sdgsdev.kemendesa.go.id/SIDcutoff/desa/{}".format(i)).json()["data"]
        for j in data_kel:
            j["idKec"] = i
            kel.append(j)
    return kel

In [43]:
kel = get_kelurahan(kec)
with open("kelurahan.json", "w") as x:
    x.write(json.dumps(kel, indent=3))

In [46]:
kel = pd.DataFrame(kel)

In [47]:
test = pd.merge(kel, kec, how="inner", left_on="idKec", right_on="id")

In [54]:
df = pd.merge(test, kab, how="inner", left_on="idKab", right_on="id")


In [56]:
df.columns = ["idKel", "namaKel", "idKec", "idKec2", "namaKec", "idKab", "idKab2", "namaKab", "idProv"]

In [58]:
df = pd.merge(df, prov, how="inner", left_on="idProv", right_on="id")

In [60]:
df = df[["idKel", "namaKel", "idKec", "namaKec", "idKab", "namaKab", "idProv", "value"]]

In [63]:
df.columns = ["idKel", "namaKel", "idKec", "namaKec", "idKab", "namaKab", "idProv", "namaProv"]

In [65]:
df.to_csv("data_desa_indonesia.csv", index=False)